<a href="https://colab.research.google.com/github/Gustavens/Google-colab-AI/blob/main/Act_3_T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Materia: PLN, Visión y Ética Computacional

Fecha: 01 de Junio de 2025

In [69]:
# ==============================================================================
# 1. INSTALACIÓN E IMPORTACIÓN DE LIBRERÍAS
# ==============================================================================
# pip install nltk

import nltk
import pprint
from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.tag import hmm
import pprint # Para imprimir las estructuras de datos de forma legible

# Descarga de paquetes de NLTK necesarios para la actividad
nltk.download('brown', quiet=True)
nltk.download('treebank', quiet=True)
nltk.download('tagsets', quiet=True)
nltk.download('universal_tagset', quiet=True)


# Inicializar PrettyPrinter para una mejor visualización de los resultados
pp = pprint.PrettyPrinter(indent=4)


In [70]:
# Descarga de paquete treebank
#Para aplicar en el modelo a entrenar con una cantidad de las primeras 3000 muestras.
nltk.download('treebank')
train_data = treebank.tagged_sents()[0:3000]
print("\n" + "="*80 + "\n")

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In [71]:
import nltk
nltk.download('tagsets_json')
print("\n" + "="*80 + "\n")

[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   Package tagsets_json is already up-to-date!


In [72]:
# ==============================================================================
# 2. EXPLORACIÓN DEL CORPUS Y TAGSETS
# ==============================================================================

print("--- 2.1. Exploración del conjunto de etiquetas Penn Treebank ---")
# La instrucción muestra las etiquetas del conjunto Penn Treebank y su significado.
nltk.help.upenn_tagset()
print("\n" + "="*80 + "\n")


print("--- 2.2. Exploración de la categoría 'news' del corpus Brown (Tagset por defecto) ---")
# Se obtienen las oraciones etiquetadas de la categoría 'news'
oraciones_news_brown = brown.tagged_sents(categories='news')
print("Ejemplo de oración etiquetada con el tagset por defecto (Penn Treebank):")
pp.pprint(oraciones_news_brown[0])
print("\n" + "="*80 + "\n")


print("--- 2.3. Exploración de la categoría 'news' con Tagset Universal ---")
# Se repite el paso anterior, pero especificando el tagset 'universal'
oraciones_news_universal = brown.tagged_sents(categories='news', tagset='universal')
print("Ejemplo de oración etiquetada con el tagset 'universal':")
pp.pprint(oraciones_news_universal[0])
print("\n" + "="*80 + "\n")

--- 2.1. Exploración del conjunto de etiquetas Penn Treebank ---
$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pa

In [73]:
# ==============================================================================
# 3. IMPLEMENTACIÓN DE LA FUNCIÓN DE DISTRIBUCIÓN DE FRECUENCIAS
# ==============================================================================

def explora_tagset_fd(genero, tagset):
    """
    Calcula la distribución de frecuencias de las etiquetas POS dentro de un género del corpus Brown.

    :parametro genero: Un genero dentro del corpus Brown
    :type genero: str o iterable(str) o None
    :parametro tagset: El nombre del tagset dentro del corpus Brown
    :type tagset: str o None (el default es 'brown')
    :return: cantidad de tipos de etiquetas, top 10 de etiquetas
    :rtype: tuple(int, list(tuple(str, int)))
    """
    print(f"--- Calculando distribución para Género: '{genero}', Tagset: '{tagset or 'default'}' ---")

    # 1. Obtener las palabras etiquetadas del corpus para el género y tagset especificados.
    palabras_etiq = brown.tagged_words(categories=genero, tagset=tagset)

    # 2. Convertir la lista de palabras etiquetadas a una lista de solo etiquetas.
    tags = [tag for (word, tag) in palabras_etiq]

    # 3. Calcular la distribución de frecuencias de las etiquetas.
    etiqFD = nltk.FreqDist(tags)

    # 4. Calcular la cantidad de tipos de etiquetas únicas.
    num_etiquetas = len(etiqFD)

    # 5. Obtener las 10 etiquetas más frecuentes y su frecuencia.
    top_tags = etiqFD.most_common(10)

    return num_etiquetas, top_tags

# --- Pruebas de la función explora_tagset_fd ---
print("--- 3.1. Probando la función explora_tagset_fd ---")

# Prueba 1: Categoría 'news' con tagset por defecto (Penn Treebank)
num_etiq_news, top_news = explora_tagset_fd('news', None)
print("Resultados para 'news' (Penn Treebank):")
print(f"Número total de tipos de etiquetas: {num_etiq_news}")
print("Top 10 etiquetas más frecuentes:")
pp.pprint(top_news)
print("-" * 40)

# Prueba 2: Categoría 'science_fiction' con tagset 'universal'
num_etiq_sf, top_sf = explora_tagset_fd('science_fiction', 'universal')
print("Resultados para 'science_fiction' (universal):")
print(f"Número total de tipos de etiquetas: {num_etiq_sf}")
print("Top 10 etiquetas más frecuentes:")
pp.pprint(top_sf)

print("\n" + "="*80 + "\n")

--- 3.1. Probando la función explora_tagset_fd ---
--- Calculando distribución para Género: 'news', Tagset: 'default' ---
Resultados para 'news' (Penn Treebank):
Número total de tipos de etiquetas: 218
Top 10 etiquetas más frecuentes:
[   ('NN', 13162),
    ('IN', 10616),
    ('AT', 8893),
    ('NP', 6866),
    (',', 5133),
    ('NNS', 5066),
    ('.', 4452),
    ('JJ', 4392),
    ('CC', 2664),
    ('VBD', 2524)]
----------------------------------------
--- Calculando distribución para Género: 'science_fiction', Tagset: 'universal' ---
Resultados para 'science_fiction' (universal):
Número total de tipos de etiquetas: 12
Top 10 etiquetas más frecuentes:
[   ('NOUN', 2747),
    ('VERB', 2579),
    ('.', 2428),
    ('DET', 1582),
    ('ADP', 1451),
    ('PRON', 934),
    ('ADJ', 929),
    ('ADV', 828),
    ('PRT', 483),
    ('CONJ', 416)]




In [74]:
# ==============================================================================
# 4. ENTRENAMIENTO Y EVALUACIÓN DE UN MODELO HMM
# ==============================================================================

print("Ayuda de la clase HiddenMarkovModelTagger ---")
# Se utiliza help para entender el funcionamiento de la clase
# help(nltk.tag.hmm.HiddenMarkovModelTagger)
print("La función help() se ha comentado para evitar un output extenso.")
print("\n" + "="*80 + "\n")

def entrenamientoHMM(oracion, tamanio):
    """
    Crea un etiquetador HMM de la categoria news del corpus Brown y se evalua con una oracion de prueba.

    :parametro oracion: Una oración sin etiquetar
    :type oracion: list(str)
    :parametro tamano: Número de oraciones para entrenar
    :type tamano: int
    :return: El objeto etiquetador, la oración de pruebas con etiquetas y la precisión (accuracy).
    :rtype: tuple(nltk.tag.hmm.HiddenMarkovModelTagger, list(tuple(str,str)), float)
    """
    oraciones_etiq = brown.tagged_sents(categories='news', tagset='universal')

    # Datos de entrenamiento
    datos_train = oraciones_etiq[:tamanio]

    # Datos de prueba (las siguientes 500 oraciones para evitar solapamiento)
    datos_test = oraciones_etiq[tamanio : tamanio + 500]

    # Entrenar el modelo HiddenMarkovModelTagger con el método train().
    tagger = nltk.tag.hmm.HiddenMarkovModelTagger.train(datos_train)

    # Utilizar el etiquetador para etiquetar la oración de prueba.
    oracion_etiqueta_hmm = tagger.tag(oracion)

    # Evaluar la precisión (accuracy) con los datos de prueba.
    acc = tagger.accuracy(datos_test)

    return tagger, oracion_etiqueta_hmm, acc

def evaluarHMM():
    """
    Función para evaluar el rendimiento del HMM con diferentes tamaños de conjunto de entrenamiento.
    """
    oraciones_etiq = brown.tagged_sents(categories='news', tagset='universal')
    # Se toma una oración de ejemplo para probar el etiquetado
    oracion_prueba = [word for word, tag in oraciones_etiq[505]]
    print(f"Oración de prueba: {' '.join(oracion_prueba)}\n")

    # --- Evaluación con 500 oraciones ---
    tagger_500, oracion_etiquetada_500, acc_500 = entrenamientoHMM(oracion_prueba, 500)
    print('--- Entrenando el modelo con 500 oraciones... ---')
    print('Oración etiquetada con el modelo:')
    pp.pprint(oracion_etiquetada_500)
    print(f'Accuracy del conjunto de pruebas: {100.0 * acc_500:.4f}%\n')

    # --- Evaluación con 3000 oraciones ---
    tagger_3000, oracion_etiquetada_3000, acc_3000 = entrenamientoHMM(oracion_prueba, 3000)
    print('--- Entrenando el modelo con 3000 oraciones... ---')
    print('Oración etiquetada con el modelo:')
    pp.pprint(oracion_etiquetada_3000)
    print(f'Accuracy del conjunto de pruebas: {100.0 * acc_3000:.4f}%')

# --- Ejecutar la evaluación del HMM ---
print("--- 4.2. Evaluación del rendimiento del Modelo HMM ---")
evaluarHMM()
print("\n" + "="*80 + "\n")


Ayuda de la clase HiddenMarkovModelTagger ---
La función help() se ha comentado para evitar un output extenso.


--- 4.2. Evaluación del rendimiento del Modelo HMM ---
Oración de prueba: He said Mitchell is against the centralization of government in Washington but looks to the Kennedy Administration for aid to meet New Jersey school and transportation crises .

--- Entrenando el modelo con 500 oraciones... ---
Oración etiquetada con el modelo:
[   ('He', 'PRON'),
    ('said', 'VERB'),
    ('Mitchell', 'NOUN'),
    ('is', 'VERB'),
    ('against', 'ADP'),
    ('the', 'DET'),
    ('centralization', 'NOUN'),
    ('of', 'ADP'),
    ('government', 'NOUN'),
    ('in', 'ADP'),
    ('Washington', 'NOUN'),
    ('but', 'CONJ'),
    ('looks', 'NOUN'),
    ('to', 'ADP'),
    ('the', 'DET'),
    ('Kennedy', 'NOUN'),
    ('Administration', '.'),
    ('for', 'ADP'),
    ('aid', 'NOUN'),
    ('to', 'PRT'),
    ('meet', 'VERB'),
    ('New', 'ADJ'),
    ('Jersey', 'NOUN'),
    ('school', 'NOUN'),
    ('

In [75]:
# ==============================================================================
# 5. MEJORA DEL MODELO HMM CON EL CORPUS TREEBANK
# ==============================================================================

print("--- 5. Mejora del Modelo con Corpus Treebank ---")

# El bajo rendimiento inicial puede deberse a la cantidad y variabilidad de los datos.
# Se utilizará el corpus Treebank, que es un estándar para el entrenamiento de etiquetadores POS.

# División de datos: 3000 para entrenamiento y el resto para pruebas.
train_data = treebank.tagged_sents()[:3000]
test_data = treebank.tagged_sents()[3000:]

print(f"Tamaño del conjunto de entrenamiento (Treebank): {len(train_data)}")
print(f"Tamaño del conjunto de pruebas (Treebank): {len(test_data)}\n")


# Entrenar un nuevo etiquetador HMM con los datos de Treebank
print("Entrenando el nuevo modelo HMM con Treebank...")
tagger_treebank = nltk.tag.hmm.HiddenMarkovModelTagger.train(train_data)
print("Modelo entrenado.\n")

# Evaluar el nuevo modelo
# Nota: El método .evaluate() está obsoleto, se usa .accuracy()
accuracy_treebank = tagger_treebank.accuracy(test_data)

print(f"Accuracy del modelo con Treebank: {accuracy_treebank*100:.4f}%")

# Probar el etiquetado en una oración de ejemplo
oracion_ejemplo = "This is a simple test for our new HMM tagger".split()
etiquetas_ejemplo = tagger_treebank.tag(oracion_ejemplo)

print("\nEjemplo de etiquetado con el modelo de Treebank:")
pp.pprint(etiquetas_ejemplo)

--- 5. Mejora del Modelo con Corpus Treebank ---
Tamaño del conjunto de entrenamiento (Treebank): 3000
Tamaño del conjunto de pruebas (Treebank): 914

Entrenando el nuevo modelo HMM con Treebank...
Modelo entrenado.

Accuracy del modelo con Treebank: 89.8424%

Ejemplo de etiquetado con el modelo de Treebank:
[   ('This', 'DT'),
    ('is', 'VBZ'),
    ('a', 'DT'),
    ('simple', 'JJ'),
    ('test', 'NN'),
    ('for', 'IN'),
    ('our', 'PRP$'),
    ('new', 'JJ'),
    ('HMM', '.'),
    ('tagger', "''")]


In [76]:
try:
    nltk.data.find('corpora/brown')
    nltk.data.find('taggers/universal_tagset')
except nltk.downloader.DownloadError:
    print("Descargando corpus 'brown' y 'universal_tagset' de NLTK...")
    nltk.download('brown', quiet=True)
    nltk.download('universal_tagset', quiet=True)
    print("Descarga completada.")

In [77]:
# ==============================================================================
# 2. FUNCIÓN PARA ENTRENAR Y EVALUAR EL ETIQUETADOR HMM
# ==============================================================================

def train_and_evaluate_hmm_brown_news():
    """
    Esta función implementa tres tareas utilizando la categoría 'news' del corpus Brown:
    1. Entrena un etiquetador HMM con el conjunto de datos de entrenamiento.
    2. Con el modelo entrenado, etiqueta una oración de ejemplo.
    3. Evalúa el modelo entrenado en el conjunto de pruebas.
    """
    print("--- 1. Carga y Preparación de Datos (Corpus Brown - Categoría 'news') ---")

    # Cargar las oraciones etiquetadas de la categoría 'news' del corpus Brown.
    # nltk.tag.hmm.HiddenMarkovModelTagger.train espera tuplas (palabra, etiqueta).
    # brown.tagged_sents() ya proporciona este formato.
    tagged_sents = brown.tagged_sents(categories='news')

    # Dividir los datos en conjuntos de entrenamiento y prueba.
    # Usaremos una división 90/10 (90% para entrenamiento, 10% para prueba).
    # La división se hace por oraciones.
    split_point = int(len(tagged_sents) * 0.9)
    train_data = tagged_sents[:split_point]
    test_data = tagged_sents[split_point:]

    print(f"Tamaño total de oraciones en 'news': {len(tagged_sents)}")
    print(f"Tamaño del conjunto de entrenamiento: {len(train_data)} oraciones")
    print(f"Tamaño del conjunto de pruebas: {len(test_data)} oraciones")

    print("\n--- 2. Entrenamiento del Etiquetador HMM ---")
    # Crear y entrenar el etiquetador HMM.
    # El método .train() aprende las probabilidades de transición y emisión
    # a partir de los datos de entrenamiento.
    tagger = hmm.HiddenMarkovModelTagger.train(train_data)
    print("Modelo HMM entrenado.")

    print("\n--- 3. Etiquetado de una Oración de Ejemplo ---")
    # Elegir una oración simple para demostrar el etiquetado.
    # ¡Importante! La función tag() espera una lista de palabras (strings), no tuplas.
    sample_sentence = "The quick brown fox jumps over the lazy dog".split()

    # Etiquetar la oración de ejemplo con el modelo entrenado.
    tagged_sample = tagger.tag(sample_sentence)

    print(f"Oración original: {' '.join(sample_sentence)}")
    print("Oración de ejemplo etiquetada por el HMM:")
    pprint.pprint(tagged_sample)

    print("\n--- 4. Evaluación del Modelo Entrenado ---")
    # Evaluar el modelo en el conjunto de pruebas.
    # El método .evaluate() calcula la precisión del etiquetador.
    # Se recomienda usar .accuracy() en versiones más recientes de NLTK.
    # La advertencia de DeprecationWarning en `evaluate()` indica que `accuracy()`
    # es la forma preferida. Sin embargo, para mantener la consistencia con ejemplos
    # que podrían usar `.evaluate()`, lo mantendremos así, pero se puede cambiar.
    accuracy = tagger.evaluate(test_data)
    print(f"Accuracy del modelo HMM en el corpus Brown 'news': {accuracy:.4f}") # Formato a 4 decimales

    print("\n--- Proceso Completado ---")


In [78]:
# ==============================================================================
# 3. EJECUCIÓN DE LA FUNCIÓN
# ==============================================================================
if __name__ == "__main__":
    train_and_evaluate_hmm_brown_news()

--- 1. Carga y Preparación de Datos (Corpus Brown - Categoría 'news') ---
Tamaño total de oraciones en 'news': 4623
Tamaño del conjunto de entrenamiento: 4160 oraciones
Tamaño del conjunto de pruebas: 463 oraciones

--- 2. Entrenamiento del Etiquetador HMM ---
Modelo HMM entrenado.

--- 3. Etiquetado de una Oración de Ejemplo ---
Oración original: The quick brown fox jumps over the lazy dog
Oración de ejemplo etiquetada por el HMM:
[('The', 'AT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'BEDZ'),
 ('jumps', 'VBN'),
 ('over', 'IN'),
 ('the', 'AT'),
 ('lazy', 'JJ'),
 ('dog', 'NN')]

--- 4. Evaluación del Modelo Entrenado ---


<ipython-input-77-9c1cecb22624>:56: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = tagger.evaluate(test_data)


Accuracy del modelo HMM en el corpus Brown 'news': 0.8506

--- Proceso Completado ---


In [79]:
import nltk
from nltk.corpus import brown
from nltk.tag import hmm
import pprint # Para imprimir las estructuras de datos de forma legible
from nltk.metrics import ConfusionMatrix # Importar ConfusionMatrix específicamente

try:
    nltk.data.find('corpora/brown')
    nltk.data.find('taggers/universal_tagset')
except nltk.downloader.DownloadError:
    print("Descargando corpus 'brown' y 'universal_tagset' de NLTK...")
    nltk.download('brown', quiet=True)
    nltk.download('universal_tagset', quiet=True)
    print("Descarga completada.")

Glosario de Acrónimos (Etiquetas POS - Corpus Brown de NLTK)

Categorías Generales:
* AT: Article (Artículo)
* NN: Noun, singular (Sustantivo, singular)
* NNS: Noun, plural (Sustantivo, plural)
* NP: Proper Noun, singular (Nombre Propio, singular)
* NPS: Proper Noun, plural (Nombre Propio, plural)
* JJ: Adjective (Adjetivo)
* JJR: Adjective, comparative (Adjetivo, comparativo)
* JJS: Adjective, superlative (Adjetivo, superlativo)
* RB: Adverb (Adverbio)
* RBR: Adverb, comparative (Adverbio, comparativo)
* RBS: Adverb, superlative (Adverbio, superlativo)
* RN: Nominal adverb (Adverbio nominal)
* RP: Adverb, particle (Partícula adverbial)
* IN: Preposition/conjunction (Preposición / Conjunción)
* CS: Conjunction, subordinating (Conjunción subordinante)
* CC: Conjunction, coordinating (Conjunción coordinante)
* TO: To (preposición "a" o marcador de infinitivo)
* VB: Verb, base form (Verbo, forma base)
* VBD: Verb, past tense (Verbo, pasado)
* VBG: Verb, gerund/present participle (Verbo, gerundio / participio presente)
* VBN: Verb, past participle (Verbo, participio pasado)
* VBP: Verb, non-3rd person singular present (Verbo, presente, no 3ª persona singular)
* VBZ: Verb, 3rd person singular present (Verbo, presente, 3ª persona singular)
* MD: Modal (Verbo modal: can, could, may, might, must, shall, should, will, would)
* FW: Foreign word (Palabra extranjera)
* EX: Existential `there` (Existencial "there" en construcciones como "there is")
* UH: Interjection (Interjección)
* SYM: Symbol (Símbolo)
* OD: Ordinal number (Número ordinal: first, second, third)

Verbos Auxiliares Específicos:
* BE: Be (forma base del verbo "ser/estar")
* BED: Be, past tense, 2nd person singular, 1st and 3rd plural (were)
* BEDZ: Be, past tense, 1st and 3rd person singular (was)
* BEG: Be, present participle (being)
* BEM: Be, present tense, 1st person singular (am)
* BEN: Be, past participle (been)
* BER: Be, present tense, 2nd person singular or 1st and 3rd plural (are)
* BEZ: Be, present tense, 3rd person singular (is)
* DO: Do (forma base del verbo "hacer/auxiliar de negación/pregunta")
* DOD: Do, past tense (did)
* DOZ: Do, present tense, 3rd person singular (does)
* HV: Have (forma base del verbo "tener/auxiliar de perfecto")
* HVD: Have, past tense (had)
* HVG: Have, present participle (having)
* HVN: Have, past participle (had)
* HVZ: Have, present tense, 3rd person singular (has)

Pronombres:
* PP$: Possessive pronoun (Pronombre posesivo: mine, yours)
* PP$$: Possessive pronoun, second (Pronombre posesivo, segunda forma: hers, his)
* PPO: Objective pronoun (Pronombre objetivo: me, him, her, us, them)
* PPS: Nominal 3rd singular non-human pronoun (Pronombre nominativo, 3ª pers. singular, no humano: it)
* PPSS: Nominal plural pronoun (Pronombre nominativo, plural: we, they)
* PN: Preposition, non-predicative (Pronombre, no predicativo - a veces se usa para 'one')
* PPL: Singular personal pronoun (Pronombre personal singular: I, you, he, she)
* PPLS: Plural personal pronoun (Pronombre personal plural: we, you, they)

Adjetivos/Adverbios Específicos:
* AP: Adjective, predicative (Adjetivo, predicativo)
* QL: Qualitative adjective (Adjetivo cualitativo)
* QLP: Qualitative adjective, plural (Adjetivo cualitativo, plural)
* QP: Quantifier (Cuantificador)
* RB$: Possessive adverb (Adverbio posesivo: e.g., here's, there's)

Wh-palabras:
* WDT: Wh-determiner (what, which, whatever)
* WP: Wh-pronoun (who, what, which)
* WPS: Possessive wh-pronoun (whose)
* WRB: Wh-adverb (where, when, why, how)

Marcadores de Formato/Estilo:
* -HL: Headline (Indica que la palabra aparece en un titular)
* -TL: Title (Indica que la palabra aparece en un título)

Puntuación:
* . : End of sentence punctuation (Puntuación de fin de oración)
* , : Comma (Coma)
* :/: : Colon/semicolon (Dos puntos / Punto y coma)
* ( ) : Parentheses (Paréntesis)
* `` : Opening quote (Comilla de apertura)
* '' : Closing quote (Comilla de cierre)

In [80]:
# ==============================================================================
# 2. FUNCIÓN PARA ENTRENAR Y EVALUAR EL ETIQUETADOR HMM (Actualizada para retornar tagger y test_data)
# ==============================================================================

def train_and_evaluate_hmm_brown_news():
    """
    Esta función entrena un etiquetador HMM con la categoría 'news' del corpus Brown,
    etiqueta una oración de ejemplo y evalúa el modelo globalmente.
    Retorna el tagger entrenado y el conjunto de datos de prueba para evaluaciones posteriores.
    """
    print("--- 1. Carga y Preparación de Datos (Corpus Brown - Categoría 'news') ---")

    tagged_sents = brown.tagged_sents(categories='news')

    # Dividir los datos en conjuntos de entrenamiento y prueba.
    # Usaremos una división 90/10 (90% para entrenamiento, 10% para prueba).
    split_point = int(len(tagged_sents) * 0.9)
    train_data = tagged_sents[:split_point]
    test_data = tagged_sents[split_point:]

    print(f"Tamaño total de oraciones en 'news': {len(tagged_sents)}")
    print(f"Tamaño del conjunto de entrenamiento: {len(train_data)} oraciones")
    print(f"Tamaño del conjunto de pruebas: {len(test_data)} oraciones")

    print("\n--- 2. Entrenamiento del Etiquetador HMM ---")
    tagger = hmm.HiddenMarkovModelTagger.train(train_data)
    print("Modelo HMM entrenado.")

    print("\n--- 3. Etiquetado de una Oración de Ejemplo (general) ---")
    sample_sentence = "The quick brown fox jumps over the lazy dog".split()
    tagged_sample = tagger.tag(sample_sentence)
    print("Oración de ejemplo (general) etiquetada:")
    pprint.pprint(tagged_sample)

    print("\n--- 4. Evaluación GLOBAL del Modelo Entrenado ---")
    # Usa tagger.accuracy(test_data) en versiones más recientes de NLTK si evaluate() da un DeprecationWarning
    accuracy = tagger.evaluate(test_data)
    print(f"Accuracy GLOBAL del modelo HMM en el corpus Brown 'news': {accuracy:.4f}")

    print("\n--- Proceso de Entrenamiento y Evaluación Global Completado ---")
    # Retornamos el tagger y los datos de prueba para usarlos en la evaluación individual
    return tagger, test_data


In [81]:
# ==============================================================================
# 3. FUNCIÓN PARA EVALUAR UNA ORACIÓN INDIVIDUAL
# ==============================================================================

def evaluate_single_sentence_with_hmm(tagger, test_data, sentence_index):
    """
    Evalúa una única oración del conjunto de pruebas utilizando un ConfusionMatrix
    y las instrucciones sugeridas en la Figura 16.

    Args:
        tagger: El modelo HMM entrenado.
        test_data: El conjunto de oraciones de prueba.
        sentence_index (int): El índice de la oración a evaluar en test_data.
    """
    print(f"\n--- Evaluación detallada para la oración con índice {sentence_index} del conjunto de pruebas ---")

    # Verificar que el índice sea válido
    if not (0 <= sentence_index < len(test_data)):
        print(f"Error: El índice {sentence_index} está fuera del rango válido "
              f"de las oraciones de prueba (0 a {len(test_data) - 1}).")
        return

    # 1. Obtener la oración real (etiquetada) del conjunto de pruebas
    real_tagged_sentence = test_data[sentence_index]

    # 2. Separar las palabras de las etiquetas reales (gold standard)
    words_to_tag = [word for word, tag in real_tagged_sentence]
    true_tags = [tag for word, tag in real_tagged_sentence]

    print(f"Oración original (palabras): {' '.join(words_to_tag)}")
    print(f"Etiquetas reales (gold standard):")
    pprint.pprint(real_tagged_sentence)

    # 3. Etiquetar la oración con el modelo HMM
    predicted_tagged_sentence = tagger.tag(words_to_tag)

    # 4. Extraer solo las etiquetas predichas
    predicted_tags = [tag for word, tag in predicted_tagged_sentence]

    print(f"Etiquetas predichas por el modelo HMM:")
    pprint.pprint(predicted_tagged_sentence)

    # 5. Crear y mostrar la matriz de confusión
    # ConfusionMatrix toma dos listas de etiquetas: (referencia verdadera, predicciones)
    print("\nMatriz de Confusión para esta oración:")
    # La matriz de confusión es más útil para evaluar el desempeño de cada etiqueta
    # sobre un conjunto más grande de datos, pero aquí se usa para visualizar
    # los aciertos y errores en esta oración específica.
    cm = ConfusionMatrix(true_tags, predicted_tags)
    print(cm)

    # Para una evaluación de precisión de esta oración específica:
    correct_tags = sum(1 for true_t, pred_t in zip(true_tags, predicted_tags) if true_t == pred_t)
    sentence_accuracy = correct_tags / len(true_tags) if true_tags else 0
    print(f"Precisión para esta oración específica: {sentence_accuracy:.4f}")

In [82]:
# ==============================================================================
# 4. EJECUCIÓN DEL FLUJO PRINCIPAL
# ==============================================================================
if __name__ == "__main__":
    # Primero, entrenamos el modelo HMM y obtenemos el tagger y los datos de prueba
    trained_hmm_tagger, test_set_data = train_and_evaluate_hmm_brown_news()

    print("\n" + "="*80)
    print(" INICIANDO EVALUACIÓN DE ORACIONES INDIVIDUALES")
    print("="*80 + "\n")

    # --- PRUEBA CON DISTINTOS ÍNDICES ---
    # Cambia estos índices para probar diferentes oraciones del conjunto de pruebas.
    # Asegúrate de que los índices sean válidos (entre 0 y len(test_set_data) - 1).
    indices_to_test = [0, 5, 76, 150, 200] # Ejemplo de índices para probar

    for idx in indices_to_test:
        evaluate_single_sentence_with_hmm(trained_hmm_tagger, test_set_data, idx)

    # --- OPCIÓN: PERMITIR AL USUARIO INTRODUCIR ÍNDICES ---
    # Descomenta el siguiente bloque si quieres probar interactivamente
    # while True:
    #     try:
    #         user_index_str = input("\nIntroduce un índice de oración para evaluar (o 'q' para salir): ")
    #         if user_index_str.lower() == 'q':
    #             break
    #         user_index = int(user_index_str)
    #         evaluate_single_sentence_with_hmm(trained_hmm_tagger, test_set_data, user_index)
    #     except ValueError:
    #         print("Entrada no válida. Por favor, introduce un número entero o 'q'.")
    #     except Exception as e:
    #         print(f"Ocurrió un error inesperado: {e}")

--- 1. Carga y Preparación de Datos (Corpus Brown - Categoría 'news') ---
Tamaño total de oraciones en 'news': 4623
Tamaño del conjunto de entrenamiento: 4160 oraciones
Tamaño del conjunto de pruebas: 463 oraciones

--- 2. Entrenamiento del Etiquetador HMM ---
Modelo HMM entrenado.

--- 3. Etiquetado de una Oración de Ejemplo (general) ---
Oración de ejemplo (general) etiquetada:
[('The', 'AT'),
 ('quick', 'JJ'),
 ('brown', 'NN'),
 ('fox', 'BEDZ'),
 ('jumps', 'VBN'),
 ('over', 'IN'),
 ('the', 'AT'),
 ('lazy', 'JJ'),
 ('dog', 'NN')]

--- 4. Evaluación GLOBAL del Modelo Entrenado ---


<ipython-input-80-aad219ff2ab1>:37: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy = tagger.evaluate(test_data)


Accuracy GLOBAL del modelo HMM en el corpus Brown 'news': 0.8506

--- Proceso de Entrenamiento y Evaluación Global Completado ---

 INICIANDO EVALUACIÓN DE ORACIONES INDIVIDUALES


--- Evaluación detallada para la oración con índice 0 del conjunto de pruebas ---
Oración original (palabras): But in all its 175 years , not a single Negro student has entered its classrooms .
Etiquetas reales (gold standard):
[('But', 'CC'),
 ('in', 'IN'),
 ('all', 'ABN'),
 ('its', 'PP$'),
 ('175', 'CD'),
 ('years', 'NNS'),
 (',', ','),
 ('not', '*'),
 ('a', 'AT'),
 ('single', 'AP'),
 ('Negro', 'NP'),
 ('student', 'NN'),
 ('has', 'HVZ'),
 ('entered', 'VBN'),
 ('its', 'PP$'),
 ('classrooms', 'NNS'),
 ('.', '.')]
Etiquetas predichas por el modelo HMM:
[('But', 'CC'),
 ('in', 'IN'),
 ('all', 'ABN'),
 ('its', 'PP$'),
 ('175', 'JJ'),
 ('years', 'NNS'),
 (',', ','),
 ('not', '*'),
 ('a', 'AT'),
 ('single', 'AP'),
 ('Negro', 'NP'),
 ('student', 'NN'),
 ('has', 'HVZ'),
 ('entered', 'VBN'),
 ('its', 'PP$'),
 ('clas

Conclusion sobre esta oracion sobre el rendimiento del modelo HMM sobre el indice 200 del conjunto de pruebas como ejemplo de análisis.

El modelo ha demostrado una gran precisión al identificar la mayoría de las palabras dentro de la oración. En particular, ha manejado con éxito etiquetas correspondientes a partes del discurso que son más comunes y predecibles, como determinantes (AT), sustantivos comunes (NN), verbos (VB, VBD), preposiciones (IN), conjunciones (CC), adverbios (RB) y pronombres (PPS, PPSS). Además, ha reconocido correctamente la puntuación básica.

Sin embargo, se identificaron errores en algunas palabras específicas:

- **Second:** La etiqueta correcta era OD-TL (Ordinal en Título), pero el modelo la clasificó como NP (Nombre Propio). Este error podría deberse a que el modelo interpretó "Second Officer" como parte de un nombre propio compuesto (ej. Norman Simmons). Otra posibilidad es que las etiquetas OD-TL sean menos frecuentes en el corpus de entrenamiento, lo que hace más difícil su correcta identificación.

- **Officer:** La etiqueta real en el corpus era NN-TL (Sustantivo en Título), pero el modelo también la clasificó como NP. Esto confirma la tendencia del modelo a agrupar palabras capitalizadas dentro de un contexto de título o nombre propio, simplificándolas como NP. Las etiquetas con el sufijo "-TL" son más específicas y pueden ser difíciles de distinguir si el modelo no ha sido expuesto a suficientes ejemplos de este tipo en el entrenamiento.

- **Simmons:** Este fue un error crítico. La etiqueta correcta era NP (Nombre Propio), pero el modelo predijo "." (Puntuación). Este tipo de fallo podría indicar que el modelo no identificó correctamente el final de la secuencia de nombre propio, o que hubo un problema en la representación contextual de la palabra. Es posible que el modelo haya esperado una puntuación al final de la oración y, por error, haya asociado esa expectativa con "Simmons".

En resumen, aunque el modelo ha aprendido bien los patrones generales del lenguaje, todavía tiene dificultades con ciertas palabras que poseen etiquetas más específicas o menos frecuentes en el corpus de entrenamiento.